In [63]:
import pandas as pd
import datetime as dt

In [64]:
pd.set_option('display.width', 5000)

In [96]:
df = pd.read_csv("schedule.csv", index_col=0)
df.head()

,День,Месяц,Год,День недели,Начало,Окончание,Дисциплина,Адрес,Аудитория,Преподаватель
0,1,9,2023,ПТ,10:45,12:15,Производственная практика: научно-исследовател...,Основной,Бета 3.1,"Овсепьян А. (Направление ""Математическое моде..."
1,1,9,2023,ПТ,13:15,14:45,Производственная практика: научно-исследовател...,Основной,Бета 3.1,"Овсепьян А. (Направление ""Математическое моде..."
2,1,9,2023,ПТ,15:00,16:30,Производственная практика: научно-исследовател...,Основной,Бета 3.1,"Овсепьян А. (Направление ""Математическое моде..."
3,2,9,2023,СБ,09:00,10:30,Производственная практика: научно-исследовател...,Основной,Бета 3.1,"Овсепьян А. (Направление ""Математическое моде..."
4,2,9,2023,СБ,10:45,12:15,Производственная практика: научно-исследовател...,Основной,Бета 3.1,"Овсепьян А. (Направление ""Математическое моде..."


In [127]:
ALL_DAYS = pd.date_range(start="08-28-2023", end="12-31-2023")
WEEKS = ALL_DAYS
WEEKS = [WEEKS[i:i+7] for i in range(0, len(WEEKS), 7)]
TODAY = dt.datetime.today().strftime("%Y-%m-%d")
CURRENT_WEEK = [week for week in WEEKS if TODAY in week][0]
CURRENT_WEEK_NUM = [NUM for NUM, week in enumerate(WEEKS) if TODAY in week][0]
NEXT_WEEK = WEEKS[CURRENT_WEEK_NUM + 1]
TIMES = {
    1: "9:00-10:30",
    2: "10:45-12:15",
    3: "13:15-14:45",
    4: "15:00-16:30",
    5: "16:45-18:15",
#     6: "18:45-20:20",
#     7: "20:25-22:00",
}
DAYS = {
    "ПН": "Понедельник",
    "ВТ": "Вторник",
    "СР": "Среда",
    "ЧТ": "Четверг",
    "ПТ": "Пятница",
    "СБ": "Суббота",
    "ВС": "Воскресенье"
}

In [128]:
schedule_template = pd.DataFrame({
    "Пары": TIMES.keys(),
    "Часы": TIMES.values()
})
schedule = pd.DataFrame({
    "Дни": ALL_DAYS,
    "День недели": list(DAYS.values()) * len(WEEKS)
}).merge(schedule_template, how='cross')

schedule["Дни"] = pd.to_datetime(schedule["Дни"])
schedule["Дисциплина"] = "  "
schedule["Аудитория"] = "  "
schedule["Преподаватель"] = "  "
schedule.head(16)

,Дни,День недели,Пары,Часы,Дисциплина,Аудитория,Преподаватель
0,2023-08-28,Понедельник,1,9:00-10:30,,,
1,2023-08-28,Понедельник,2,10:45-12:15,,,
2,2023-08-28,Понедельник,3,13:15-14:45,,,
3,2023-08-28,Понедельник,4,15:00-16:30,,,
4,2023-08-28,Понедельник,5,16:45-18:15,,,
5,2023-08-29,Вторник,1,9:00-10:30,,,
6,2023-08-29,Вторник,2,10:45-12:15,,,
7,2023-08-29,Вторник,3,13:15-14:45,,,
8,2023-08-29,Вторник,4,15:00-16:30,,,
9,2023-08-29,Вторник,5,16:45-18:15,,,


In [133]:
for date in ALL_DAYS:
    day = date.day
    month = date.month
    year = date.year
    
    day_info = df.query("День == @day & Месяц == @month & Год == @year")
    
    if day_info.empty:
        continue
    else:
        for i, row in day_info.iterrows():
            day_of_week = DAYS[row["День недели"]]
            start = row["Начало"]
            end = row["Окончание"]
            disp = row["Дисциплина"]
            address = row["Адрес"]
            room = row["Аудитория"]
            teacher = row["Преподаватель"]
            
            clear_teachers = []
            for one_teacher in teacher.split(","):
                clear_teachers.append(one_teacher.split("(")[0].strip().replace("  ", " "))
            clear_teachers = ", ".join(clear_teachers)
            
            lesson_description = f"{disp}\n{clear_teachers}\nауд. {room} ({address})"
            
            schedule.loc[
                (schedule["День недели"] == day_of_week) 
                & 
                (schedule["Дни"] == pd.to_datetime(date))
                &
                (
                    (schedule["Часы"].str.contains(start))
                    |
                    (schedule["Часы"].str.contains(end))
                ), 
                "Преподаватель"
            ] = clear_teachers
            
            schedule.loc[
                (schedule["День недели"] == day_of_week) 
                & 
                (schedule["Дни"] == pd.to_datetime(date))
                &
                (
                    (schedule["Часы"].str.contains(start))
                    |
                    (schedule["Часы"].str.contains(end))
                ), 
                "Аудитория"
            ] = room
            
            schedule.loc[
                (schedule["День недели"] == day_of_week) 
                &
                (schedule["Дни"] == pd.to_datetime(date))
                &
                (
                    (schedule["Часы"].str.contains(start))
                    |
                    (schedule["Часы"].str.contains(end))
                ), 
                "Дисциплина"
            ] = disp

schedule["Дни"] = schedule["Дни"].astype(str)
schedule["День недели"] = schedule["Дни"] + " (" + schedule["День недели"] + ")"
schedule = schedule.drop("Дни", axis=1)

In [134]:
schedule = schedule.set_index(["День недели", "Часы"])

In [136]:
schedule.to_html("sirius.html", na_rep=" ", justify='center', border=3)